In [137]:
# links

# acs
# https://api.census.gov/data/2018/acs/acs5/profile/examples.html
# https://api.census.gov/data/2018/acs/acs5/profile/geography.html
# https://api.census.gov/data/2018/acs/acs5/variables.html
# https://data.census.gov/cedsci/table?q=United%20States
# https://www.census.gov/data/developers/data-sets/acs-5year.html
# https://api.census.gov/data/2018/acs/acs5/subject.html

# qwi
# https://www.census.gov/data/developers/data-sets/qwi.html
# https://api.census.gov/data/timeseries/qwi/sa/variables.html
# https://api.census.gov/data/timeseries/qwi/sa/examples.html
# https://api.census.gov/data/timeseries/qwi.html
# https://lehd.ces.census.gov/doc/QWIPU_Data_Schema.pdf
# https://lehd.ces.census.gov/doc/QWI_101.pdf

In [138]:
%pwd

'C:\\Users\\Tola\\Documents\\Github\\capstone'

In [139]:
%cd Documents\Github\capstone

[WinError 3] The system cannot find the path specified: 'Documents\\Github\\capstone'
C:\Users\Tola\Documents\Github\capstone


In [140]:
#manipulate dataframes/geodataframes in python
import pandas as pd
import geopandas as gpd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

#for parsing the urls

In [141]:
import numpy as np
from shapely.geometry import Point
from fiona.crs import from_epsg

In [142]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
from ipywidgets import Button 
import ipywidgets as widgets

In [143]:
#drawing basic map
center = (40.7210907,-73.9877836)
basemap = basemap_to_tiles(basemaps.CartoDB.Positron)

icon = Icon(icon_url='icon.png', icon_size=[15, 15])
marker = Marker(location=center, draggable=False, icon=icon, opacity = 0)

m = Map(layers=(basemap, ), center=center, zoom=15, min_zoom = 11, max_zoom = 20)

###HALF-MI BUFFER
#set location of center as location of the point
markerlocation=marker.location
    
radius = 402 #in-meters
color = "black"
fill_color = "white"
fill_opacity = .2
weight = 2

##click on map to add marker, removes old marker as well
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        layer_group.clear_layers();
        marker = Marker(location=kwargs.get('coordinates'), draggable=False, icon=icon, opacity = 0)
        layer_group.add_layer(marker)
        
        #updating the circle's location
        markerlocation=marker.location
        circle.location = markerlocation
        layer_group.add_layer(circle);

        ########################### adding the black overlay
        df2 = pd.DataFrame(markerlocation)

        df=df2.transpose()
        df.columns=['Latitude','Longitude']

        #creating a geodataframe from the point
        gdf = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
        #creating circle for cut
        # gdf.crs = 'EPSG:4326'
        gdf.crs = from_epsg(3857)
        test=gdf.copy()
        #making buffer
        test['geometry'] = test.geometry.buffer(.004,  cap_style=1, join_style=1)
        # change back
        test.crs = from_epsg(3857)
        #creating rectangle for cut
        rect_orig=gdf.copy()
        rect_orig['geometry'] = rect_orig.geometry.buffer(4,  cap_style=1, join_style=1)
        #difference
        diff = gpd.overlay(rect_orig, test, how='difference')

        geo_data = GeoData(geo_dataframe = diff,
                       style={'fillColor': 'black', 'color':'black', 'weight':2,'fillOpacity':0.5},
    #                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Boundary')
        layer_group.add_layer(geo_data)
        ################
        
m.on_interaction(handle_click)
######

#create half-mi buffer
circle = Circle()
circle.location = markerlocation
circle.radius = radius
circle.color = color
circle.fill_color = fill_color
circle.fill_opacity = fill_opacity
circle.weight = weight
circle.opacity = 0

###########

    
# def on_location_changed(event):
# #remove any circles existing first
#     layer_group.clear_layers();

# #update location of center as location of the point
#     markerlocation = marker.location
#     circle.location = markerlocation
#     layer_group.add_layer(circle);
        
# pass
# marker.observe(on_location_changed, 'location')
# #######


#adding button that performs the analysis on click 
button = Button(description="Analyze")

def on_button_clicked(b):   
    #extracting the marker location for the buffer
    location=circle.location
    
    df2 = pd.DataFrame(location)

    df=df2.transpose()
    df.columns=['Latitude','Longitude']

    #creating a geodataframe from the point
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
#creating circle for cut
    # gdf.crs = 'EPSG:4326'
    gdf.crs = from_epsg(3857)
    test=gdf.copy()
    #making buffer
    test['geometry'] = test.geometry.buffer(.004,  cap_style=1, join_style=1)
    # change back
    test.crs = from_epsg(3857)
#creating rectangle for cut
    rect_orig=gdf.copy()
    rect_orig['geometry'] = rect_orig.geometry.buffer(4,  cap_style=1, join_style=1)
#difference
    diff = gpd.overlay(rect_orig, test, how='difference')

    geo_data = GeoData(geo_dataframe = diff,
                   style={'fillColor': 'black', 'color':'black', 'weight':2,'fillOpacity':0.5},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')
    layer_group.add_layer(geo_data)
    
button.on_click(on_button_clicked)

    
widget_control = WidgetControl(widget=button, position='topright')
m.add_control(widget_control)


layer_group = LayerGroup(layers=(marker, circle))
m.add_layer(layer_group)

display(m)

Map(center=[40.7210907, -73.9877836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [144]:
#Extracting latitude and longitude values as separate strings from marker location ie from center
lat = str(markerlocation[0])
lon = str(markerlocation[1])
print(lat,'+', lon)

40.7210907 + -73.9877836


In [145]:
#Figure out which state and census block a latitude/longitude is in
#USING PARAMETERS 

#Encode parameters 
# lat = '42.3295'
# lon = '-71.0826'

#Contruct request URL
url = f'https://geo.fcc.gov/api/census/block/find?'

# api call using variables instead of directly is way easier
data_url = f'{url}lat={lat}&lon={lon}&format=json'
#Get response from API
response=requests.get(data_url)

#Parse json in response
data = response.json()

print(response.text)

#Print FIPS code
print(data['results'][0]['state_fips'])
print(data['results'][0]['county_fips'])
print(data['results'][0]['county_name'])

{"input":{"lat":40.7210907,"lon":-73.9877836},"results":[{"block_fips":"360610030012000","bbox":[-73.988354,40.719833,-73.98695,40.721234],"county_fips":"36061","county_name":"New York","state_fips":"36","state_code":"NY","state_name":"New York","block_pop_2015":504,"amt":"AMT001","bea":"BEA010","bta":"BTA321","cma":"CMA001","eag":"EAG001","ivm":"IVM001","mea":"MEA002","mta":"MTA001","pea":"PEA001","rea":"REA001","rpc":"RPC001","vpc":"VPC001"},{"block_fips":"360610030012001","bbox":[-73.989114,40.720077,-73.987769,40.721462],"county_fips":"36061","county_name":"New York","state_fips":"36","state_code":"NY","state_name":"New York","block_pop_2015":531,"amt":"AMT001","bea":"BEA010","bta":"BTA321","cma":"CMA001","eag":"EAG001","ivm":"IVM001","mea":"MEA002","mta":"MTA001","pea":"PEA001","rea":"REA001","rpc":"RPC001","vpc":"VPC001"}]}
36
36061
New York


In [146]:
#Extracting the county and state being studied 
state = data['results'][0]['state_fips']
county_fips = data['results'][0]['county_fips']

print(state)
print(county_fips)

#Separating out the county fips from the state
county = county_fips[2:]

print(county)

36
36061
061


In [147]:
#DOWNLOADING ACS 

# https://api.census.gov/data/2018/acs/acs5/profile?get=DP04_0001E&for=tract:*&in=state:36%20county:005,047&key=9330dc4bf086a84f19fb412bb15f232507301de6
api_key = '9330dc4bf086a84f19fb412bb15f232507301de6'
acs_url = f'https://api.census.gov/data/2018/acs/acs5/'

pop_sex = 'B01003_001E,B01001_002E,B01001_026E'
household_type = 'B11001_002E,B11001_007E'
education = 'B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E'
household_income = 'B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E'
means_of_transport = 'B08301_002E,B08301_003E,B08301_003E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E'
travel_time = 'B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E'

# api call using variables instead of directly is way easier
pop_sex_url = f'{acs_url}?&get={pop_sex}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
household_type_url = f'{acs_url}?&get={household_type}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
education_url = f'{acs_url}?&get={education}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
household_income_url = f'{acs_url}?&get={household_income}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
means_of_transport_url = f'{acs_url}?&get={means_of_transport}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
travel_time_url = f'{acs_url}?&get={travel_time}&for=tract:*&in=state:{state}%20county:*&key={api_key}'

response_acs1=requests.get(pop_sex_url)
response_acs2=requests.get(household_type_url)
response_acs3=requests.get(education_url)
response_acs4=requests.get(household_income_url)
response_acs5=requests.get(means_of_transport_url)
response_acs6=requests.get(travel_time_url)
# print(response_acs.text)

#load the response into a JSON
data_acs1=response_acs1.json()
data_acs2=response_acs2.json()
data_acs3=response_acs3.json()
data_acs4=response_acs4.json()
data_acs5=response_acs5.json()
data_acs6=response_acs6.json()

#ignoring the first element which is column names, replacing columns with the columns from the table
acs1=pd.DataFrame(data_acs1[1:], columns=data_acs1[0])
acs2=pd.DataFrame(data_acs2[1:], columns=data_acs2[0])
acs3=pd.DataFrame(data_acs3[1:], columns=data_acs3[0])
acs4=pd.DataFrame(data_acs4[1:], columns=data_acs4[0])
acs5=pd.DataFrame(data_acs5[1:], columns=data_acs5[0])
acs6=pd.DataFrame(data_acs6[1:], columns=data_acs6[0])

acs1.head()
acs2.head()
acs3.head()
acs4.head()
acs5.head()
acs6.head()

,B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E,state,county,tract
0,9,79,226,362,159,64,100,74,218,192,74,48,36,091,061404
1,49,186,218,233,149,45,149,75,30,205,170,47,36,095,740500
2,65,67,41,87,45,22,116,48,49,56,57,23,36,089,492900
3,102,298,261,196,239,57,168,24,12,50,68,14,36,089,492100
4,148,485,404,419,719,411,557,147,182,270,89,60,36,091,062405


In [7]:
#manipulate dataframes/geodataframes in python
import pandas as pd
import geopandas as gpd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

#for parsing the urls

In [8]:
#DOWNLOADING ACS 
#TESTINGGGGG
state=36

api_key = '9330dc4bf086a84f19fb412bb15f232507301de6'
acs_url = f'https://api.census.gov/data/2018/acs/acs5/'

variables = 'B01003_001E,B01001_002E,B01001_026E,B11001_002E,B11001_007E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,B08301_002E,B08301_003E,B08301_003E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E,B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E'

# api call using variables instead of directly is way easier
variables_url = f'{acs_url}?&get={variables}&for=tract:*&in=state:{state}%20county:*&key={api_key}'


data_acs=requests.get(variables_url).json()


#ignoring the first element which is column names, replacing columns with the columns from the table
acs=pd.DataFrame(data_acs[1:], columns=data_acs[0])


acs.head()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [148]:
#Downloading census 1

# api_key = 9330dc4bf086a84f19fb412bb15f232507301de6
with open('census_key.txt') as key:
    api_key=key.read().strip()
 
# here is defining the variables...
time='2019-Q2'

MSA_short='metropolitan%20statistical%20area/micropolitan%20statistical%20area'
qwi_url = f'https://api.census.gov/data/timeseries/qwi/sa'

# 	https://api.census.gov/data/timeseries/qwi/sa?get=year,agegrp,quarter&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&in=state:24&time=2010-Q1&key=YOUR_KEY_GOES_HERE
sex='sex'
age_group='agegrp'
earnings='EarnBeg'
employment='Emp'
industry='industry'

# api call using variables instead of directly is way easier
# selecting industry by SECTOR to make shit way easier
sex_url = f'{qwi_url}?&get={sex}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&key={api_key}'
age_group_url = f'{qwi_url}?&get={age_group}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&key={api_key}'
earnings_url = f'{qwi_url}?&get={earnings}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&key={api_key}'
employment_url = f'{qwi_url}?&get={employment}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&key={api_key}'
industry_url = f'{qwi_url}?&get={industry}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&industry=11&industry=21&industry=22&industry=23&industry=31-33&industry=42&industry=44-45&industry=48-49&industry=51&industry=52&industry=53&industry=54&industry=55&industry=56&industry=61&industry=62&industry=71&industry=72&industry=81&industry=92&key={api_key}'

response_qwi1=requests.get(sex_url)
response_qwi2=requests.get(age_group_url)
response_qwi3=requests.get(earnings_url)
response_qwi4=requests.get(employment_url)
response_qwi5=requests.get(industry_url)
# print(response_qwi.text)

#load the response into a JSON
data_qwi1=response_qwi1.json()
data_qwi2=response_qwi2.json()
data_qwi3=response_qwi3.json()
data_qwi4=response_qwi4.json()
data_qwi5=response_qwi5.json()

#ignoring the first element which is just column names, replacing columns with the columns from the table
qwi1=pd.DataFrame(data_qwi1[1:], columns=data_qwi1[0])
qwi2=pd.DataFrame(data_qwi2[1:], columns=data_qwi2[0])
qwi3=pd.DataFrame(data_qwi3[1:], columns=data_qwi3[0])
qwi4=pd.DataFrame(data_qwi4[1:], columns=data_qwi4[0])
qwi5=pd.DataFrame(data_qwi5[1:], columns=data_qwi5[0])

qwi4.tail()
# qwi1['industry'].dtypes
# qwi1['industry'].unique

,Emp,time,state,metropolitan statistical area/micropolitan statistical area
22,517744,2019-Q2,36,40380
23,11901,2019-Q2,36,42900
24,303056,2019-Q2,36,45060
25,122801,2019-Q2,36,46540
26,34799,2019-Q2,36,48060


In [149]:
#write about this in paper as limitations - dataset is not the most reliable 
print(len(qwi1))
print(len(qwi2))
print(len(qwi3))
print(len(qwi4)) #27 because aggregated already to totals in each MSA
print(len(qwi5))

81
243
27
27
539


In [150]:
print(len(acs1))
print(len(acs2))
print(len(acs3))
print(len(acs4))
print(len(acs5))

4918
4918
4918
4918
4918


In [153]:
#rename headings, concatenate datasets, summarize etc
#CENSUS TRACTS

acs1.rename(columns = {'B01003_001E':'pop',\
                       'B01001_002E':'male',\
                       'B01001_026E':'female'\
                      }, inplace = True) 

acs2.rename(columns = {'B11001_002E':'family',\
                       'B11001_007E':'nonfamily'\
                      }, inplace = True) 

# acs3["lessthan9thgrade"]=acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']
# acs3["9thto12thgrade"]= acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']+acs3['']
acs3["highschoolgraduate_GED"]= acs3['B15003_017E']+acs3['B15003_018E']
acs3["some_college"]= acs3['B15003_019E']+acs3['B15003_020E']
acs3["associate_degree"] = acs3['B15003_021E']
acs3["bachelors_degree"] = acs3['B15003_022E']
acs3["graduate_or_professional_degree"] = acs3['B15003_023E']+acs3['B15003_024E']
acs3['doctoratedegree'] = acs3["B15003_025E"]
acs3.drop(['B15003_001E','B15003_017E','B15003_018E','B15003_019E','B15003_020E','B15003_021E','B15003_022E','B15003_023E','B15003_024E','B15003_025E'], axis = 1) 

acs4["lessthan10000"]= acs4['B19001_002E']
acs4["10000to34999"]= acs4['B19001_003E']+acs4['B19001_004E']+acs4['B19001_005E']+acs4['B19001_006E']+acs4['B19001_007E']
acs4["35000to49999"] = acs4['B19001_008E']+acs4['B19001_009E']+acs4['B19001_010E']
acs4["50000to74999"] = acs4['B19001_011E']+acs4['B19001_011E']
acs4["75000to99000"] = acs4['B19001_013E']
acs4['100000to199000'] = acs4["B19001_014E"]+acs4['B19001_015E']+acs4['B19001_016E']
acs4['200000ormore'] = acs4["B19001_017E"]
acs4.drop(['B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'])

acs5.rename(columns = {'B08301_003E':'car_truck_van_drove_alone',\
                       'B08301_004E':'car_truck_van_carpool',\
                       'B08301_010E':'public_transportation',\
                       'B08301_016E':'taxicab',\
                       'B08301_017E':'motorcycle',\
                       'B08301_018E':'bicycle',\
                       'B08301_019E':'walked',\
                       'B08301_021E':'worked_at_home',\
                       'B08301_020E':'other_means'\
                      }, inplace = True) 

acs6["lessthan10mins"]= acs6['B08303_002E']+acs6['B08303_003E']
acs6["10to29mins"]= acs6['B08303_004E']+acs6['B08303_005E']+acs6['B08303_006E']+acs6['B08303_007E']
acs6["30to59mins"]= acs6['B08303_008E']+acs6['B08303_009E']+acs6['B08303_010E']+acs6['B08303_011E']
acs6["60minsormore"]= acs6['B08303_012E']+acs6['B08303_013E']
acs6.drop(['B08303_002E','B08303_003E','B08303_004E','B08303_005E','B08303_006E','B08303_007E','B08303_008E','B08303_009E','B08303_010E','B08303_011E','B08303_012E','B08303_013E'])

KeyError: "['B08303_002E' 'B08303_003E' 'B08303_004E' 'B08303_005E' 'B08303_006E'\n 'B08303_007E' 'B08303_008E' 'B08303_009E' 'B08303_010E' 'B08303_011E'\n 'B08303_012E' 'B08303_013E'] not found in axis"

In [174]:
qwi1.rename(columns = {'metropolitan statistical area/micropolitan statistical area':'msa'}, inplace = True) 
qwi1

,sex,time,state,msa
0,0,2019-Q2,36,10580
1,1,2019-Q2,36,10580
2,2,2019-Q2,36,10580
3,0,2019-Q2,36,11220
4,1,2019-Q2,36,11220
...,...,...,...,...
76,1,2019-Q2,36,46540
77,2,2019-Q2,36,46540
78,0,2019-Q2,36,48060
79,1,2019-Q2,36,48060


In [203]:
#Downloading census 1

# api_key = 9330dc4bf086a84f19fb412bb15f232507301de6
with open('census_key.txt') as key:
    api_key=key.read().strip()
 
# here is defining the variables...
time='2019-Q2'

MSA_short='metropolitan%20statistical%20area/micropolitan%20statistical%20area'
qwi_url = f'https://api.census.gov/data/timeseries/qwi/sa'

# 	https://api.census.gov/data/timeseries/qwi/sa?get=year,agegrp,quarter&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&in=state:24&time=2010-Q1&key=YOUR_KEY_GOES_HERE
sex='sex'
age_group='agegrp'
earnings='EarnBeg'
employment='Emp'
# industry='industry'

# api call using variables instead of directly is way easier
# selecting industry by SECTOR to make shit way easier
# all_data = f'{qwi_url}?&get=sex,agegrp,EarnBeg,Emp&for={MSA_short}:*&in=state:{state}&time={time}&key={api_key}'
all_data = f'{qwi_url}?&get=sex,agegrp,EarnBeg,Emp,industry&for={MSA_short}:*&in=state:{state}&time={time}&industry=11&industry=21&industry=22&industry=23&industry=31-33&industry=42&industry=44-45&industry=48-49&industry=51&industry=52&industry=53&industry=54&industry=55&industry=56&industry=61&industry=62&industry=71&industry=72&industry=81&industry=92&key={api_key}'
# industry_url = f'{qwi_url}?&get={industry}&for={MSA_short}:*&for=county:*&in=state:{state}&time={time}&industry=11&industry=21&industry=22&industry=23&industry=31-33&industry=42&industry=44-45&industry=48-49&industry=51&industry=52&industry=53&industry=54&industry=55&industry=56&industry=61&industry=62&industry=71&industry=72&industry=81&industry=92&key={api_key}'

response_all=requests.get(all_data)
# response_qwi5=requests.get(industry_url)
# print(response_qwi.text)

#load the response into a JSON
data_all=response_all.json()
# data_qwi5=response_qwi5.json()

#ignoring the first element which is just column names, replacing columns with the columns from the table
qwi_all=pd.DataFrame(data_all[1:], columns=data_all[0])
# qwi5=pd.DataFrame(data_qwi5[1:], columns=data_qwi5[0])

qwi_all.sort_values('metropolitan statistical area/micropolitan statistical area', ascending=False).head()
# qwi1['industry'].dtypes
# qwi1['industry'].unique

,sex,agegrp,EarnBeg,Emp,industry,time,industry,state,metropolitan statistical area/micropolitan statistical area
14552,2,A08,2116,117,92,2019-Q2,92,36,48060
14197,2,A04,2021,540,44-45,2019-Q2,44-45,36,48060
14183,0,A08,2160,309,44-45,2019-Q2,44-45,36,48060
14184,1,A00,3090,2469,44-45,2019-Q2,44-45,36,48060
14185,1,A01,696,61,44-45,2019-Q2,44-45,36,48060


In [208]:
# qwi1.sort_values('metropolitan statistical area/micropolitan statistical area', ascending=False).head()

#AGGREGATE QWI by MSA
# print(qwi1['metropolitan statistical area/micropolitan statistical area'].unique)
#new df with only 
selected = (qwi1[qwi1.sex == '1'])
selected.head()

#Make new columns for male and female
#count the occurence of sex = 1 for male

#count the occurence of sex = 2 for female

# len(df[df['education'] == '9th'])
# (df[education]=='9th').sum()


,sex,time,state,msa
1,1,2019-Q2,36,10580
4,1,2019-Q2,36,11220
7,1,2019-Q2,36,12180
10,1,2019-Q2,36,12860
13,1,2019-Q2,36,13780


In [166]:
#METROPILITAN STATISTICAL AREAS
#sum up each by the metropolitan statistical area before concatenate
#separate into different dataframes with the MSAs 
#then combine these into one dataframe 
#then concatenate 

#MICROPOLITAN STATISTICAL AREAS
qwi1

,sex,time,state,metropolitan statistical area/micropolitan statistical area
0,0,2019-Q2,36,10580
1,1,2019-Q2,36,10580
2,2,2019-Q2,36,10580
3,0,2019-Q2,36,11220
4,1,2019-Q2,36,11220
...,...,...,...,...
76,1,2019-Q2,36,46540
77,2,2019-Q2,36,46540
78,0,2019-Q2,36,48060
79,1,2019-Q2,36,48060


In [98]:
#proportional split
#join census tracts, msas to census data


#calculate area of original census geographies


#clip original to circle


#calculate new census values in area by multiplying 

In [99]:
#NOW we can begin text generation 

In [103]:
qwi2['agegrp'].unique

<bound method Series.unique of 0      A00
1      A01
2      A02
3      A03
4      A04
      ... 
238    A04
239    A05
240    A06
241    A07
242    A08
Name: agegrp, Length: 243, dtype: object>

In [104]:
qwi1['sex'].unique
#0=total

<bound method Series.unique of 0     0
1     1
2     2
3     0
4     1
     ..
76    1
77    2
78    0
79    1
80    2
Name: sex, Length: 81, dtype: object>

In [105]:
qwi5['industry'].unique

AttributeError: 'DataFrame' object has no attribute 'unique'

In [106]:
#Don't discount the firm size then...could just be displaying as none 
qwi5

,industry,time,industry,state,metropolitan statistical area/micropolitan statistical area
0,11,2019-Q2,11,36,10580
1,21,2019-Q2,21,36,10580
2,22,2019-Q2,22,36,10580
3,23,2019-Q2,23,36,10580
4,31-33,2019-Q2,31-33,36,10580
...,...,...,...,...,...
534,62,2019-Q2,62,36,48060
535,71,2019-Q2,71,36,48060
536,72,2019-Q2,72,36,48060
537,81,2019-Q2,81,36,48060


In [107]:
qwi1

,sex,time,state,metropolitan statistical area/micropolitan statistical area
0,0,2019-Q2,36,10580
1,1,2019-Q2,36,10580
2,2,2019-Q2,36,10580
3,0,2019-Q2,36,11220
4,1,2019-Q2,36,11220
...,...,...,...,...
76,1,2019-Q2,36,46540
77,2,2019-Q2,36,46540
78,0,2019-Q2,36,48060
79,1,2019-Q2,36,48060


In [ ]:
df['sum'] = df[list(df.columns)].sum(axis=1)
